# Project

## Libraries

In [2]:
import requests
import json
import time
import pandas as pd
from tqdm import tqdm
import psycopg2
import yaml

# from pydantic import BaseModel, ValidationError, validator (## for the future)

/Users/yeungadrian/Documents/repo/ML-Ops/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Riot Games API

In [3]:
with open("../config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile)

/var/folders/8g/67y55mms29zf6yt6q92_3hsr0000gn/T/ipykernel_52336/1863735473.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [6]:
api_key = cfg['riotgamesapi']['key']

In [ ]:
region = 'euw1'

In [8]:
challenger_url = f'https://{region}.api.riotgames.com/tft/league/v1/challenger?api_key={api_key}'
challenger_response = requests.get(url = challenger_url).json()

In [27]:
#Writes json response into a file called api_data.json
with open("api_data.json", "w") as write_file:
    json.dump(challenger_response, write_file)

#Writes json response into a dataframe
challenger_response_df = pd.DataFrame(challenger_response['entries'])


In [31]:
challenger_response_df

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,ST93EcMRl_XieHvdxVA_-3evb8WUZxeejm1BNzQkZALg3vU,SNG Lev Trotskij,1730,I,40,183,True,False,False,False
1,DAUPNC0PRLyXCiGvx4WPhOE9SlZHmdZjYigR7EMTr4edePA,ANO shreddin,1031,I,54,241,False,False,True,False
2,mCuEgOhd-ucdIGJI-1dR9ZqfRd08Ie9Dyrc-HcbXGqeQ8N7m,DV1 Sage,1057,I,47,248,False,False,True,False
3,PxGBPFGbeK3ZadtEk6WHoLlvk_bBnFyZB7gKESZcUmJnluyk,ARMA ACKK,1784,I,69,263,True,False,False,False
4,njcn3SSmte_vUrSw2yHu_HWnB_a-5dz1ghb7LbRk739p5-_k,BigDaddyLL,880,I,42,277,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
195,WagQouAo01fQ7lEzJ8IaV9ytKCUagyfOtoXAzMVG-aN1RAo,Lorus,978,I,44,317,True,False,False,False
196,qdIRHPCsg2P-BxS276kAJR8WRsEyRycO9bBXKLUy1ENzSPY,Senjó,824,I,38,328,False,False,True,False
197,L_bYU8q50LMN9Zf3SnBkX19vSTu0kKNdhKw4JtkQuiJWMhQ,iG Stelfalador,1205,I,49,216,True,False,False,False
198,-zbpNbiD3FBBB0bzMidm76FPx1sNO8f-BZxIETXDfiZlYJMC,AieZen,1006,I,23,130,False,False,False,False


In [30]:
# entries is just a key of the json
challenger_response.keys()

dict_keys(['tier', 'leagueId', 'queue', 'name', 'entries'])

## Postgresql Database

In [8]:
params = cfg['postgresql']
# Connect to the PostgreSQL database
conn = psycopg2.connect(**params, port = 5431)
# Create a new cursor - A PostgreSQL database cursor is a read-only pointer that allows a program, regardless of the language used, to access the result set of a query. 
cur = conn.cursor()
# A function that takes in a PostgreSQL query and outputs a pandas database 

In [ ]:


class UserModel(BaseModel):
    summonerId: str
    leaguepoints: int

In [36]:
#Loops through rows for df
for i in tqdm(range(0, challenger_response_df.shape[0])):
    summonerId = challenger_response_df["summonerId"][i]
    # do all the columns that are called TBD

    cur.execute(f'''
    INSERT INTO public."Users"(
	    id, "summonerName", "summonerId", "rankedLeague", puuid, leaguepoints, region)
	    VALUES (DEFAULT, 'TBD', 'TBD', 'CHALLENGER', 'Temporary' 'TBD', {region})
        WHERE 
        ''')

    conn.commit()


KeyboardInterrupt: 

In [9]:
#Use this cell to delete table data, although ID counter needs to be reset manually in Postgres
cur.execute(f'''
DELETE FROM public."Users"
''')

In [12]:
conn.commit()

# Try...
#Incorrect, then rollback transaction

Commit or roll-back...

for loop:
take API and put into SQL 
Execute 1 person at a time...
Validation of data type...

psychopg to insert data into database

## Prefect

In [13]:
import prefect
from prefect import task, Flow

@task
def hello_task():
    logger = prefect.context.get("logger")
    logger.info("Hello world!")

with Flow("hello-flow") as flow:
    hello_task()

flow.run()

[2021-09-29 22:06:01+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'hello-flow'
[2021-09-29 22:06:01+0100] INFO - prefect.TaskRunner | Task 'hello_task': Starting task run...
[2021-09-29 22:06:01+0100] INFO - prefect.hello_task | Hello world!
[2021-09-29 22:06:01+0100] INFO - prefect.TaskRunner | Task 'hello_task': Finished task run for task with final state: 'Success'
[2021-09-29 22:06:01+0100] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">

In [14]:
from prefect import task

@task
def extract():
    """Get a list of data"""
    return [1, 2, 3]

@task
def transform(data):
    """Multiply the input by 10"""
    return [i * 10 for i in data]

@task
def load(data):
    """Print the data to indicate it was received"""
    print("Here's your data: {}".format(data))


In [20]:
from prefect import task, Flow, Parameter


@task(log_stdout=True)
def say_hello(name):
    print("Hello, {}!".format(name))


with Flow("My First Flow") as flow:
    name = Parameter('name')
    say_hello(name)


flow.run(name='world') # "Hello, world!"
flow.run(name='Marvin') # "Hello, Marvin!"

[2021-09-29 22:15:05+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'My First Flow'
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'name': Starting task run...
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'name': Finished task run for task with final state: 'Success'
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'say_hello': Starting task run...
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Hello, world!
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'say_hello': Finished task run for task with final state: 'Success'
[2021-09-29 22:15:05+0100] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2021-09-29 22:15:05+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'My First Flow'
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'name': Starting task run...
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'name': Finished task run for task with final state: 'Suc

<Success: "All reference tasks succeeded.">